# House Prices – Advanced Regression Project

In diesem Notebook entwickle ich ein Regressionsmodell zur Vorhersage von Immobilienpreisen. 
Der Fokus liegt auf einer vollständigen ML‑Pipeline mit EDA, Feature Engineering, Modellvergleich, 
SHAP‑Interpretation und Residuenanalyse.


## **Daten laden und Überblick**

In [ ]:
import pandas as pd 
import numpy as np 

df = pd.read_csv("data\train.csv")
df.head() 
df.info() 
df = df.drop(columns=['Id'])
df.isna().sum().sort_values(ascending=False).head(20)

### Datenüberblick

Hier werden die Daten geladen und eine rster Überblick über Struktur, Datentypen und fehlende Werte gegeben.

## **Explorative Datenanalyse (EDA)**

### Zielvariable

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(df['SalePrice'], kde=True)
plt.title("Verteilung von SalePrice")
plt.show()

### Log-Transformation

In [ ]:
df['SalePrice_log'] = np.log1p(df['SalePrice'])
sns.histplot(df['SalePrice_log'], kde=True)
plt.title("Log-transformierte Zielvariable")
plt.show()

### Korrelationen

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(numeric_only=True)['SalePrice'].sort_values(ascending=False).to_frame(), annot=True)
plt.title("Korrelationen mit SalePrice")
plt.show()

Die wichtigsten Einflussfaktoren werden visuell untersucht, um Muster und Zusammhänge zu erkennen.

## **Feature Engineering**

In [ ]:
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
df['HouseAge'] = df['YrSold'] - df['YearBuilt']
df['RemodAge'] = df['YrSold'] - df['YearRemodAdd']
df['QualityIndex'] = df['OverallQual'] * df['OverallCond']
df['TotalBath'] = df['FullBath'] + 0.5 * df['HalfBath'] + df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath']
df['PorchSF'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
df['GarageScore'] = df['GarageCars'] * df['GarageArea']

Es wurden mehrere neue Features erstellt, welche auf Domain-Wissen und Kaggle-Best-Pratices basieren.

## **Preprocessing**
- Missing Values
- One-Hot_Encoding
- Train/Test-Split

In [ ]:
# Kategorien, bei denen NaN bedeutet: Feature existiert NICHT
cols_none = ['PoolQC', 'Alley', 'Fence', 'MiscFeature', 'FireplaceQu']
df[cols_none] = df[cols_none].fillna('None')

# Numerische Features, bei denen NaN = 0 sinnvoll ist
cols_zero = ['GarageArea', 'GarageCars', 'TotalBsmtSF', 'BsmtFinSF1']
df[cols_zero] = df[cols_zero].fillna(0)

# Kategorien, bei denen NaN = häufigster Wert (Mode)
df['MasVnrType'] = df['MasVnrType'].fillna(df['MasVnrType'].mode()[0])
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

# LotFrontage abhängig vom Stadtteil (Neighborhood) imputieren
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median())
)

# Weitere sinnvolle Imputationen
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)


# Ordinale Kategorien in Zahlen umwandeln
qual_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}

df['ExterQual'] = df['ExterQual'].map(qual_map)
df['KitchenQual'] = df['KitchenQual'].map(qual_map)


# One-Hot-Encoding für alle übrigen kategorialen Variablen
df = pd.get_dummies(df, drop_first=True)


# Train/Test-Split
from sklearn.model_selection import train_test_split

X = df.drop(['SalePrice', 'SalePrice_log'], axis=1)
y = df['SalePrice_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## **Modellvergleich**
Es wurden 3 Gradient-Boosting-Modelle trainiert und verglichen: XGBoost, LightGBM und CatBoost.
Alle Modelle wurden mit ähnlichen Hyperparametern konfiguriert, um einen fairen Vergleich zu ermöglichen.

Die Bewertung erfolgte anhand von: 
- **RMSE** (Root Mean Squared Error) 
- **R²** (Bestimmtheitsmaß)

CatBoost erzielte die beste Performance und wurde daher als finales Modell ausgewählt.
### Evaluationsfunktion

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def evaluate(model_name, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name}: RMSE = {rmse:.4f}, R² = {r2:.4f}")
    return rmse, r2

### XGBoost

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

rmse_xgb, r2_xgb = evaluate("XGBoost", y_test, y_pred_xgb)

### LightGBM

In [ ]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbosity=0
)

lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)

rmse_lgbm, r2_lgbm = evaluate("LightGBM", y_test, y_pred_lgbm)

### CatBoost

In [ ]:
from catboost import CatBoostRegressor

cat = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    loss_function='RMSE',
    verbose=False,
    random_seed=42
)

cat.fit(X_train, y_train)
y_pred_cat = cat.predict(X_test)

rmse_cat, r2_cat = evaluate("CatBoost", y_test, y_pred_cat)

In [ ]:
results = pd.DataFrame({
    "Modell": ["XGBoost", "LightGBM", "CatBoost"],
    "RMSE": [rmse_xgb, rmse_lgbm, rmse_cat],
    "R²": [r2_xgb, r2_lgbm, r2_cat]
})

results

## **SHAP-Analyse**
Um zu verstehen, welche Features den größten Einfluss auf die Vorhersagen des Modells haben, wird SHAP (SHapley Additive exPlanations) verwendet. SHAP ermöglicht eine transparente Interpretation von Gradient-Boosting-Modellen und zeigt, welche Merkmale den Preis erhöhen oder senken. Die Analyse basiert auf dem finalen CatBoost-Modell.

In [ ]:
import shap

explainer = shap.TreeExplainer(cat)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

## **Residuenanalyse**
Um zu überprüfen, ob das Modell systematische Fehler macht, werden die Residuen analysiert. Die Residuenanalyse zeigt, ob das Modell gut kalibriert ist, ob Ausreißer existieren und ob bestimmte Preisbereiche schlechter vorhergesagt werden.
Untersucht werden: 
- Residuen vs. Vorhersagen 
- Histogramm der Residuen 
- QQ‑Plot (Normalverteilungstest) 
- Tatsächliche vs. vorhergesagte Werte

In [ ]:
residuals = y_test - y_pred_cat

plt.figure(figsize=(8,5))
sns.scatterplot(x=y_pred_cat, y=residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Vorhergesagte Werte")
plt.ylabel("Residuen")
plt.title("Residuen vs. Vorhersagen")
plt.show()

Punkte sollten zufällig um die 0-Linie streuen

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(residuals, kde=True)
plt.title("Histogramm der Residuen")
plt.xlabel("Residuen")
plt.show()

Starke Ausreßer wären ein Problem

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("QQ-Plot der Residuen")
plt.show()

Residuen folgen nicht perfektn einer Normalverteilung, aber es gibt keinen extremen Abweichungen.

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x=y_test, y=y_pred_cat, alpha=0.6)
plt.xlabel("Tatsächliche Werte")
plt.ylabel("Vorhergesagte Werte")
plt.title("Tatsächliche vs. vorhergesagte Werte")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()

Punkte sollten nahe der roten Linie liegen.

### Fazit der Residuenanalyse

- Die Residuen streuen zufällig um die 0-Linie → keine systematischen Verzerrungen.
- Das Histogramm zeigt eine leichte Schiefe, was bei Immobilienpreisen typisch ist.
- Der QQ-Plot bestätigt, dass die Residuen nicht perfekt normalverteilt sind, 
  was bei Gradient-Boosting-Modellen normal ist.
- Die Vorhersagen folgen der Idealgeraden gut, mit etwas Streuung bei hohen Preisen.

Insgesamt zeigt die Residuenanalyse, dass das Modell stabil ist und gut generalisiert.


## **9. Finales Modell speichern**

Das finale Modell ist der CatBoostRegressor, da er im Modellvergleich die beste Performance erzielt hat.
Um das Modell später wiederverwenden zu können (z. B. für Deployment, erneute Vorhersagen oder eine
Kaggle-Submission), wird es im CatBoost-eigenen Format `.cbm` gespeichert.


In [ ]:
cat.save_model("models\catboost_final_model.cbm") 
print("Modell erfolgreich gespeichert.")

## **10. Fazit**

In diesem Projekt wurde ein vollständiger Machine-Learning-Workflow zur Vorhersage von Immobilienpreisen umgesetzt. 
Die Pipeline umfasst Explorative Datenanalyse, systematisches Feature Engineering, Preprocessing, Modellvergleich, 
SHAP-Interpretation und Residuenanalyse.

Der Modellvergleich zeigte, dass der CatBoostRegressor die beste Performance erzielt und sowohl robuste als auch 
interpretierbare Ergebnisse liefert. Die SHAP-Analyse bestätigte, dass Wohnfläche, Bauqualität, Kellerfläche und 
Renovierungsjahr zu den wichtigsten Einflussfaktoren gehören. Die Residuenanalyse zeigte keine systematischen 
Verzerrungen und deutete auf ein gut kalibriertes Modell hin.

Insgesamt liefert das finale Modell zuverlässige Vorhersagen und bildet die zentralen Zusammenhänge des 
Immobilienmarkts realistisch ab. Das Projekt zeigt, wie Feature Engineering, Modellinterpretation und 
Validierung zusammenwirken, um ein leistungsfähiges und nachvollziehbares Regressionsmodell zu entwickeln.
